In [ ]:
# Viggy Kumaresan and Prajwal Vijendra
# Wharton Case Competition


In [1]:
# Import datasets
import pandas as pd
import numpy as np
import seaborn as sns
import os

act_clients_train = pd.read_csv("ACTIVE_CLIENTS_TRAIN.csv") # active client

client_demog = pd.read_csv("CLIENT_DEMOGRAPHICS_TRAIN.csv") # client demo

nfp_agency_train = pd.read_csv("NFP_AGENCY_TRAIN.csv") # agency

nfp_nurse_info_train = pd.read_csv("NFP_NURSE_INFO_TRAIN.csv") # nurse info

nurse_tenure_train = pd.read_csv("NURSE_TENURE_TRAIN.csv") # nurse tenure

print(act_clients_train.head())

   study_id   casenumber  month  site_id  time_in_program  visit_count  \
0  47821000  CS_47822000      1      109              422            2   
1  41958000  CS_41959000      1      109              731            1   
2  15068000  CS_15069000      1      109               15            2   
3  34967000  CS_34968000      1      109              438            2   
4    101000    CS_102000      1      109              369            2   

  program_phase  days_last_visit  nurse_id  nurse_id_assigned  
0   Toddlerhood               10      6165               6165  
1   Toddlerhood               29      6165               6165  
2     Pregnancy                5      6165               6165  
3       Infancy                3      6165               6165  
4       Infancy                4      6165               6165  


In [2]:
act_clients_train.shape

(227206, 10)

In [3]:
act_clients_train['nurse_id'].nunique()

1403

In [4]:
client_demog.shape

(32032, 47)

In [5]:
client_demog['casenumber'].nunique()

31842

In [6]:
# Merging
# Within Client
client_master = pd.merge(client_demog,act_clients_train, on = ['study_id', 'casenumber'] )
client_master.head()

,study_id,casenumber,reasonfordismissal,length_of_stay,visit_count_x,transfer,gestation_at_enrollment,enrolled_after_birth,still_enrolled,client_age,...,asq_referral,asq_at_least_once,month,site_id,time_in_program,visit_count_y,program_phase,days_last_visit,nurse_id,nurse_id_assigned
0,17499000,CS_17500000,Child reached 2nd birthday,960,45,0,18.428571,0.0,0,NaN,...,0.0,1.0,1,169,466,1,Infancy,10,21712,21712
1,17499000,CS_17500000,Child reached 2nd birthday,960,45,0,18.428571,0.0,0,NaN,...,0.0,1.0,2,169,496,2,Infancy,0,21712,21712
2,17499000,CS_17500000,Child reached 2nd birthday,960,45,0,18.428571,0.0,0,NaN,...,0.0,1.0,3,169,527,1,Toddlerhood,17,21712,21712
3,17499000,CS_17500000,Child reached 2nd birthday,960,45,0,18.428571,0.0,0,NaN,...,0.0,1.0,4,169,558,1,Toddlerhood,24,21712,21712
4,17499000,CS_17500000,Child reached 2nd birthday,960,45,0,18.428571,0.0,0,NaN,...,0.0,1.0,5,169,587,1,Toddlerhood,7,21712,21712


In [7]:
client_master.shape

(227206, 55)

NOTE: visit_count_x is visits over whole program, visit_count_y is visits per month (which we will use for our modeling)

In [8]:
list(client_master.columns.values)

['study_id',
 'casenumber',
 'reasonfordismissal',
 'length_of_stay',
 'visit_count_x',
 'transfer',
 'gestation_at_enrollment',
 'enrolled_after_birth',
 'still_enrolled',
 'client_age',
 'preterm',
 'initiated_bf',
 'bmi_cat',
 'weight_gain',
 'nicu_stay',
 'immunization6mo',
 'immunization12mo',
 'homeless_any',
 'educationlvlbline',
 'educationlvlbline2',
 'edenrollmentbline',
 'edenrollmshsbline',
 'changeineducation',
 'employmentbline',
 'employmenthrsbline',
 'changeinemployment',
 'maritalstatus',
 'race',
 'ethnicity',
 'raceethnicity',
 'raceethnicity2',
 'subsequentpreg18mo',
 'smokespreg',
 'substanceusepreg',
 'alcoholusepreg',
 'medicaid_any',
 'schip_any',
 'indian_health_any',
 'wic_any',
 'ccap_any',
 'tanf_any',
 'ssi_any',
 'special_needs_any',
 'early_intervention_child_any',
 'foodstamp_any',
 'asq_referral',
 'asq_at_least_once',
 'month',
 'site_id',
 'time_in_program',
 'visit_count_y',
 'program_phase',
 'days_last_visit',
 'nurse_id',
 'nurse_id_assigned']

In [9]:
nurse_tenure_train.head()

,nurse_id,month,tenure
0,9655,1,2192.0
1,24216,1,871.0
2,25880,1,582.0
3,39515,1,-289.0
4,22371,1,1053.0


In [10]:
# Merging
# Merging within nurse
nurse_master = pd.merge(nfp_nurse_info_train,nurse_tenure_train, on = ['nurse_id','month'])
nurse_master.head()

,nurse_id,month,site_id,first_visit_dt,fte,term_flag,nurse_primary_role,numcases_actual,numcases_assigned,tenure
0,6165,1,109,30-May-06,1.0,NaN,Nurse Home Visitor,24.0,24.0,3441.0
1,6165,2,109,30-May-06,1.0,NaN,Nurse Home Visitor,25.0,25.0,3471.0
2,6165,3,109,30-May-06,1.0,NaN,Nurse Home Visitor,24.0,24.0,3502.0
3,6165,4,109,30-May-06,1.0,NaN,Nurse Home Visitor,25.0,25.0,3533.0
4,6165,5,109,30-May-06,1.0,NaN,Nurse Home Visitor,26.0,25.0,3562.0


In [11]:
# Merge client with nurse
nurse_client_master = pd.merge(client_master,nurse_master, on = ['site_id', 'nurse_id', 'month'] )
nurse_client_master.head(25)

,study_id,casenumber,reasonfordismissal,length_of_stay,visit_count_x,transfer,gestation_at_enrollment,enrolled_after_birth,still_enrolled,client_age,...,days_last_visit,nurse_id,nurse_id_assigned,first_visit_dt,fte,term_flag,nurse_primary_role,numcases_actual,numcases_assigned,tenure
0,17499000,CS_17500000,Child reached 2nd birthday,960,45,0,18.428571,0.0,0,NaN,...,10,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
1,30767000,CS_30768000,Child reached 2nd birthday,955,55,0,NaN,0.0,0,NaN,...,9,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
2,31520000,CS_31521000,Unable to locate,543,23,0,22.142857,0.0,0,NaN,...,40,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
3,28235000,CS_28236000,Unable to locate,460,18,0,28.857143,0.0,0,NaN,...,88,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
4,3399000,CS_3400000,Child reached 2nd birthday,895,32,0,23.714286,0.0,0,15.0,...,3,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
5,26265000,CS_26266000,Moved out of service area,692,28,0,13.571429,0.0,0,16.0,...,18,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
6,30059000,CS_30060000,Moved out of service area,121,4,0,NaN,NaN,0,16.0,...,23,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
7,28877000,CS_28878000,Child reached 2nd birthday,917,35,0,13.285714,0.0,0,17.0,...,12,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
8,31923000,CS_31924000,Child reached 2nd birthday,943,26,0,NaN,0.0,0,17.0,...,172,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0
9,187000,CS_188000,Child reached 2nd birthday,964,42,0,9.571429,0.0,0,18.0,...,32,21712,21712,4-Sep-12,1.0,NaN,Nurse Home Visitor,22.0,22.0,1152.0


In [12]:
nurse_client_master[(nurse_client_master['nurse_id'] == 21712) & (nurse_client_master['casenumber'] == 'CS_17500000')].T

,0,22,44,67,90,112,134,158,180,205,229,253
study_id,17499000,17499000,17499000,17499000,17499000,17499000,17499000,17499000,17499000,17499000,17499000,17499000
casenumber,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000,CS_17500000
reasonfordismissal,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday,Child reached 2nd birthday
length_of_stay,960,960,960,960,960,960,960,960,960,960,960,960
visit_count_x,45,45,45,45,45,45,45,45,45,45,45,45
transfer,0,0,0,0,0,0,0,0,0,0,0,0
gestation_at_enrollment,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286,18.4286
enrolled_after_birth,0,0,0,0,0,0,0,0,0,0,0,0
still_enrolled,0,0,0,0,0,0,0,0,0,0,0,0
client_age,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
list(nurse_client_master.columns.values)

['study_id',
 'casenumber',
 'reasonfordismissal',
 'length_of_stay',
 'visit_count_x',
 'transfer',
 'gestation_at_enrollment',
 'enrolled_after_birth',
 'still_enrolled',
 'client_age',
 'preterm',
 'initiated_bf',
 'bmi_cat',
 'weight_gain',
 'nicu_stay',
 'immunization6mo',
 'immunization12mo',
 'homeless_any',
 'educationlvlbline',
 'educationlvlbline2',
 'edenrollmentbline',
 'edenrollmshsbline',
 'changeineducation',
 'employmentbline',
 'employmenthrsbline',
 'changeinemployment',
 'maritalstatus',
 'race',
 'ethnicity',
 'raceethnicity',
 'raceethnicity2',
 'subsequentpreg18mo',
 'smokespreg',
 'substanceusepreg',
 'alcoholusepreg',
 'medicaid_any',
 'schip_any',
 'indian_health_any',
 'wic_any',
 'ccap_any',
 'tanf_any',
 'ssi_any',
 'special_needs_any',
 'early_intervention_child_any',
 'foodstamp_any',
 'asq_referral',
 'asq_at_least_once',
 'month',
 'site_id',
 'time_in_program',
 'visit_count_y',
 'program_phase',
 'days_last_visit',
 'nurse_id',
 'nurse_id_assigned',
 '

In [14]:
nfp_nurse_info_train['numcases_actual'].value_counts()

24.0    944
23.0    884
22.0    827
21.0    813
25.0    810
20.0    774
19.0    681
18.0    653
1.0     568
17.0    541
26.0    517
15.0    502
16.0    487
14.0    470
13.0    408
2.0     397
12.0    360
10.0    321
27.0    303
4.0     292
3.0     283
11.0    274
9.0     266
8.0     263
5.0     262
7.0     257
6.0     252
28.0    128
29.0     60
30.0     36
31.0     17
32.0     10
33.0      2
36.0      2
37.0      1
34.0      1
Name: numcases_actual, dtype: int64

In [15]:
nurse_client_master.shape

(227153, 62)

In [16]:
list(nurse_client_master.columns.values)

['study_id',
 'casenumber',
 'reasonfordismissal',
 'length_of_stay',
 'visit_count_x',
 'transfer',
 'gestation_at_enrollment',
 'enrolled_after_birth',
 'still_enrolled',
 'client_age',
 'preterm',
 'initiated_bf',
 'bmi_cat',
 'weight_gain',
 'nicu_stay',
 'immunization6mo',
 'immunization12mo',
 'homeless_any',
 'educationlvlbline',
 'educationlvlbline2',
 'edenrollmentbline',
 'edenrollmshsbline',
 'changeineducation',
 'employmentbline',
 'employmenthrsbline',
 'changeinemployment',
 'maritalstatus',
 'race',
 'ethnicity',
 'raceethnicity',
 'raceethnicity2',
 'subsequentpreg18mo',
 'smokespreg',
 'substanceusepreg',
 'alcoholusepreg',
 'medicaid_any',
 'schip_any',
 'indian_health_any',
 'wic_any',
 'ccap_any',
 'tanf_any',
 'ssi_any',
 'special_needs_any',
 'early_intervention_child_any',
 'foodstamp_any',
 'asq_referral',
 'asq_at_least_once',
 'month',
 'site_id',
 'time_in_program',
 'visit_count_y',
 'program_phase',
 'days_last_visit',
 'nurse_id',
 'nurse_id_assigned',
 '

## Train Logistic Regression Model

In [17]:
# Only select the variables we want and put into new dataframe
nurse_training = nurse_client_master[['numcases_actual', 'month','length_of_stay',
'visit_count_y',
'transfer',
'gestation_at_enrollment',
'enrolled_after_birth',
'still_enrolled',
'client_age',
'bmi_cat',
'weight_gain',
'nicu_stay',
'immunization6mo',
'immunization12mo',
'homeless_any',
'edenrollmentbline',
'employmentbline',
'maritalstatus',
'race',
'ethnicity',
'subsequentpreg18mo',
'smokespreg', 
'substanceusepreg',
'alcoholusepreg',
'medicaid_any',      
'schip_any', 
'indian_health_any',
'wic_any', 
'ccap_any',
'tanf_any',
'ssi_any',
'special_needs_any',
'early_intervention_child_any',
'foodstamp_any',
'asq_referral',
'asq_at_least_once',
'program_phase',
'days_last_visit',
'preterm',
'initiated_bf',
'changeineducation',
'changeinemployment',
'reasonfordismissal', 
'fte'
]]
# 'nurse_id', 
# 'casenumber'

In [18]:
# Create outcome variable from reasonfordismissal
pos = ['Graduated','Child reached 2nd birthday','Client received what she needs from the program','Client returned to work or school','Child reached 26 months','Child reached 2nd Birthday','Client returned to work','Client returned to school']

neu = ['Moved out of service area','Unable to contact','Unable to locate','Excessive missed appointments/attempted visits','Agency Closure','Child no longer in family custody','Unable to accommodate requested schedule','Nurse resigned/no room in other nurse caseloads','Pressure from family','Client incarcerated','Team Closure','Safety of the nurse','Unable to serve client due to language','Child no longer in familys custody (parental righ','Nurse Resigned and no room in remaining nurses ca','Client transferred','Nurse resigned and no room in remaining nurses ca','Enrolled in NFP','O-Excessive missed appointments/attempted visits','Client Transferred','O-Client incarcerated','O-Already enrolled in another program','Waiting list']
neg = ['Refused new nurse','Client is receiving services from another program','Miscarried/fetal death','Infant death','Refused Participation','Client refused continuation following CWS report','Dissatisfied with program','Maternal death','Client refused NFP continuation following report t','Refused participation','O-Dissatisfied with program','O-Maternal death']

In [19]:
nurse_training['reasonfordismissal'] = nurse_training['reasonfordismissal'].str.replace('\'','')

/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
nurse_training['reasonfordismissal'] = nurse_training['reasonfordismissal'].replace(pos,1)

nurse_training['reasonfordismissal'] = nurse_training['reasonfordismissal'].replace(neg,0)
nurse_training['reasonfordismissal'] = nurse_training['reasonfordismissal'].replace(neu,1)

/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [21]:
nurse_training['program_phase'] = nurse_training['program_phase'].replace('Infancy',0)

nurse_training['program_phase'] = nurse_training['program_phase'].replace('Toddlerhood',1)
nurse_training['program_phase'] = nurse_training['program_phase'].replace('Pregnancy',2 )

/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [22]:
nurse_training.shape

(227153, 44)

In [24]:
# Chose to drop rows with any empty values, since we can't use this for Logistic Regression
nurse_omit_2 = nurse_training.dropna()
nurse_omit_2.shape

(98786, 44)

In [51]:
# Class imbalance
nurse_omit_2['reasonfordismissal'].value_counts()

1.0    96508
0.0     2278
Name: reasonfordismissal, dtype: int64

In [25]:
# Import sklearn train_test_split to split our dataset into training and testing datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(nurse_omit_2.iloc[:,:-3], nurse_omit_2.iloc[:,42], test_size=0.2, random_state=42)

In [26]:
# Mean-center our numcases_actual predictor, since this is what we ultimately want to modify/optimize
X_train['numcases_actual'] = X_train['numcases_actual'] - (X_train['numcases_actual'].mean())

In [27]:
X_train['numcases_actual'].mean()

3.2776782193823014e-15

In [28]:
# Since our classes are imbalanced, we used imblearn to perform Synthetic Mintory Oversampling (SMOTE)
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [44]:
# Import Logistic Regression module from sklearn, fit classifier and predict on our test set
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_res, y_res)

probs = clf.predict(X_test)

/Users/vigneshkumaresan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
# Total Accuracy
clf.score(X_test, y_test)

0.8898167830752101

In [46]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, probs)

array([[  431,    46],
       [ 2131, 17150]])

In [47]:
# AUC 
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, probs)

0.8965203140968119

In [48]:
# Precision
from sklearn.metrics import precision_score
precision_score(y_test, probs, average='micro') 

0.8898167830752101

In [49]:
# Print out coefficients
# This is what we are really interested in, since we want to see the log_odds effect of numcases_actual, compared to our mean numcases_actual
cdf = pd.DataFrame(clf.coef_.T, X_train.columns, columns=['Coefficients'])
print(cdf)

                              Coefficients
numcases_actual                  -0.013439
month                            -0.013773
length_of_stay                    0.019503
visit_count_y                    -0.075086
transfer                          3.126732
gestation_at_enrollment           0.119487
enrolled_after_birth              0.000000
still_enrolled                    0.000000
client_age                        0.024054
bmi_cat                          -0.190557
weight_gain                      -0.011464
nicu_stay                         0.472289
immunization6mo                  -0.209333
immunization12mo                  0.603393
homeless_any                     -0.147522
edenrollmentbline                 0.029842
employmentbline                  -0.340479
maritalstatus                    -0.227005
race                              0.018392
ethnicity                         0.027362
subsequentpreg18mo               -0.086836
smokespreg                        1.162524
substanceus

In [50]:
import math
math.exp(-0.013439)

0.9866509001870174